In [31]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import hvplot.pandas
from datetime import datetime, timedelta
import plotly.express as px
import altair as alt
from vega_datasets import data
import pandas_datareader as web

In [32]:
initial_investment = 10000

In [33]:
tickers_df = portfolio_df = pd.DataFrame({"date": pd.date_range(start='2023-01-01', end='2023-05-21', freq='B')})
tickers_df['AAPL'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df['TSLA'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df['AMD'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df.set_index('date', inplace=True)
tickers_df

,AAPL,TSLA,AMD
date,,,
2023-01-02,65,87,60
2023-01-03,74,43,87
2023-01-04,68,94,34
2023-01-05,33,44,35
2023-01-06,2,57,59
...,...,...,...
2023-05-15,54,1,52
2023-05-16,92,50,37
2023-05-17,87,20,12


In [34]:
tickers = ["AAPL", "TSLA", "DIS"]
weights = [(1 / len(tickers))] * len(tickers)

In [62]:
def cumulative_returns(df, weights, investment):
    pct_change = df.pct_change()
    added_weights = pct_change.dot(weights)
    calculation = (1 + added_weights).cumprod()
    profit = (investment * calculation)
    profit_df = pd.DataFrame(profit)
    profit_df.columns = ['profit']
    return profit_df

In [63]:
user_cr = cumulative_returns(tickers_df, weights, initial_investment)
user_cr

,profit
date,
2023-01-02,NaN
2023-01-03,10275.71
2023-01-04,11973.85
2023-01-05,7913.88
2023-01-06,8024.08
...,...
2023-05-15,inf
2023-05-16,inf
2023-05-17,inf


In [37]:
def covariance(df, ticker, market):
    covariance_rolling = df[ticker].rolling(window=21).cov(df[market])
    rolling_cov_df = pd.DataFrame(covariance_rolling)
    drop_na_df = rolling_cov_df.dropna()
    return drop_na_df

In [38]:
user_covariance = covariance(tickers_df, 'AAPL', 'TSLA')
user_covariance

,0
date,
2023-01-30,90.64
2023-01-31,99.70
2023-02-01,153.30
2023-02-02,23.83
2023-02-03,23.98
...,...
2023-05-15,-110.49
2023-05-16,-121.44
2023-05-17,-179.75


In [39]:
def variance(df, market):
    rolling_variance = df[market].rolling(window=21).var()
    rolling_variance_df = pd.DataFrame(rolling_variance)
    rolling_variance_df = rolling_variance_df.dropna()
    return rolling_variance_df

In [40]:
user_variance = variance(tickers_df, 'AAPL')
user_variance

,AAPL
date,
2023-01-30,537.33
2023-01-31,553.86
2023-02-01,575.65
2023-02-02,609.36
2023-02-03,600.26
...,...
2023-05-15,573.66
2023-05-16,608.76
2023-05-17,648.95


In [41]:
def daily_drawdown(df):
    roll_max = df.cummax()
    roll_min = df.cummin()
    daily_drawdown = round(((roll_max - roll_min) / roll_max)*100, 2)
    return daily_drawdown

In [42]:
user_daily_drawdown = daily_drawdown(tickers_df)
user_daily_drawdown

,AAPL,TSLA,AMD
date,,,
2023-01-02,0.00,0.00,0.00
2023-01-03,12.16,50.57,31.03
2023-01-04,12.16,54.26,60.92
2023-01-05,55.41,54.26,60.92
2023-01-06,97.30,54.26,60.92
...,...,...,...
2023-05-15,100.00,100.00,98.98
2023-05-16,100.00,100.00,98.98
2023-05-17,100.00,100.00,98.98


In [43]:
max_user_drawdown = max(user_daily_drawdown)
max_user_drawdown

'TSLA'

In [44]:
# def tracking_error(df, tickers, market):
#     track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
#     track_error_df = pd.DataFrame(track_error)
#     return track_error

In [45]:
def tracking_error(df, tickers, market):
    trackingerror_i = (df[tickers] - df[market]).rolling(window=21).std()
    trackingerror_df = trackingerror_i.to_frame()
    trackingerror_df = trackingerror_df.dropna()
    trackingerror_df.columns = ['tracking error']
    return trackingerror_df

In [46]:
user_tracking_error = tracking_error(tickers_df, 'AAPL', 'TSLA')
user_tracking_error

,tracking error
date,
2023-01-30,36.57
2023-01-31,36.31
2023-02-01,35.62
2023-02-02,39.28
2023-02-03,39.20
...,...
2023-05-15,39.11
2023-05-16,39.86
2023-05-17,42.44


In [47]:
def beta(covariance, variance):
    return covariance / variance

In [48]:
user_beta = beta(user_covariance, user_variance)
user_beta

,0,AAPL
date,,
2023-01-30,NaN,NaN
2023-01-31,NaN,NaN
2023-02-01,NaN,NaN
2023-02-02,NaN,NaN
2023-02-03,NaN,NaN
...,...,...
2023-05-15,NaN,NaN
2023-05-16,NaN,NaN
2023-05-17,NaN,NaN


In [49]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [50]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
user_sharpe_ratio

AAPL   29.53
TSLA   28.23
AMD    27.80
dtype: float64

In [51]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

In [52]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,profit
date,
2023-01-02,716665.67
2023-01-03,49449999.00
2023-01-04,3280183332.33
2023-01-05,125740361110.11
2023-01-06,5071527898147.15
...,...
2023-05-15,59730014609766375366532499526457311261095791054...
2023-05-16,36236208863258266303893786291055958492682577948...
2023-05-17,14736058271058359919357982031312605726797100156...


In [53]:
def portfolio_distribution_chart(tickers, weights):
    chart = px.pie(values=weights, names=tickers, hole=.5)
    return chart

In [54]:
user_portfolio_distribution = portfolio_distribution_chart(tickers, weights)
user_portfolio_distribution

In [55]:
def user_tracking_error_chart(df):
    fig = px.line(df, x=df.index, y=df.columns)
    return fig

In [56]:
user_te_chart = user_tracking_error_chart(user_tracking_error)
user_te_chart

In [60]:
def user_cumulative_return_chart(df):
    fig = px.line(df, x=df.index, y=df.columns)
    return fig

In [61]:
user_cr_chart = user_cumulative_return_chart(user_cr)
user_cr_chart